In [3]:
#tianjianmeng ml212jt

In [193]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import matplotlib.pyplot as plt
from nltk import FreqDist
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import re
import nltk
import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import networkx as nx
from transformers import BartTokenizer, BartForConditionalGeneration
import spacy
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### Web scraping

In [23]:
html_my = """<tbody>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ProfilePic.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ProfilePic.jpg&amp;jpegquality=80" alt="Dr. Noorhan Abbas">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8807/dr-noorhan-abbas">Abbas</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Noorhan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 0084</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.H.Abbas@leeds.ac.uk">N.H.Abbas@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research &amp; Teaching Fellow </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/13233/professor-isolde-adler">Adler</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Isolde</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6819</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:I.M.Adler@leeds.ac.uk">I.M.Adler@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Visiting Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__3__4.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__3__4.png&amp;jpegquality=80" alt="Dr Ban Al-Jassani">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/mechanical-engineering/staff/12626/dr-ban-al-jassani">Al-Jassani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Ban Adil Naji</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.A.N.Al-Jassani@leeds.ac.uk">B.A.N.Al-Jassani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow/Teaching Assistant </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Bogdan_Alecu_photo_square_400.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Bogdan_Alecu_photo_square_400.jpg&amp;jpegquality=80" alt="Dr Bogdan Alecu">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10225/dr-bogdan-alecu">Alecu</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Bogdan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.Alecu@leeds.ac.uk">B.Alecu@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_4.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_4.png&amp;jpegquality=80" alt="Dr Sharib Ali">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11465/dr-sharib-ali">Ali</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sharib</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.S.Ali@leeds.ac.uk">S.S.Ali@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=alsalka-mhd-ammar.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=alsalka-mhd-ammar.jpg&amp;jpegquality=80" alt="Dr Mohammad Ammar Alsalka">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/635/dr-mohammad-ammar-alsalka">Alsalka</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mhd Ammar</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1142</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.A.Alsalka@leeds.ac.uk">M.A.Alsalka@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AAltahhan_3.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AAltahhan_3.jpg&amp;jpegquality=80" alt="Dr Abdulrahman Altahhan">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8784/dr-abdulrahman-altahhan">Altahhan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Abdulrahman</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1310</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Altahhan@leeds.ac.uk">A.Altahhan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Teaching Fellow in Artificial Intelligence </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=atwell-eric.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=atwell-eric.jpg&amp;jpegquality=80" alt="Professor Eric Atwell">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/33/professor-eric-atwell">Atwell</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Eric</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5761</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:E.S.Atwell@leeds.ac.uk">E.S.Atwell@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Artificial Intelligence for Language</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__70__1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__70__1.png&amp;jpegquality=80" alt="Dr Zeeshan">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12946/dr-zeeshan">Babar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Muhammad Zeeshan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Z.Babar@leeds.ac.uk">M.Z.Babar@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=1638730846808.jpeg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=1638730846808.jpeg&amp;jpegquality=80" alt="Dr Fatemeh Banaie">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10151/dr-fatemeh-banaie">Banaie Heravan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Fatemeh</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:F.BanaieHeravan@leeds.ac.uk">F.BanaieHeravan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lowe-amy.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lowe-amy.jpg&amp;jpegquality=80" alt="Dr Amy Lowe">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6763/dr-amy-lowe">Beloe</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Amy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Beloe@leeds.ac.uk">A.Beloe@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=brandonbennett.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=brandonbennett.jpg&amp;jpegquality=80" alt="Dr Brandon Bennett">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/121/dr-brandon-bennett">Bennett</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Brandon</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1070</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.Bennett@leeds.ac.uk">B.Bennett@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=WIN_20220530_13_01_33_Pro.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=WIN_20220530_13_01_33_Pro.jpg&amp;jpegquality=80" alt="Harriet Bennett">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/13223/harriet-bennett">Bennett</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Harriet</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.G.Bennett@leeds.ac.uk">H.G.Bennett@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Student Education Service Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image_250x302x.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image_250x302x.png&amp;jpegquality=80" alt="Dr. Markus Billeter">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8810/dr-markus-billeter">Billeter</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Markus</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Billeter@leeds.ac.uk">M.Billeter@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Assistant Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Peter_Bollada.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Peter_Bollada.jpg&amp;jpegquality=80" alt="Dr Peter Bollada">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/523/dr-peter-bollada">Bollada</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Peter</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2567</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:P.C.Bollada@leeds.ac.uk">P.C.Bollada@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10418/amy-brereton">Brereton</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Amy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.M.Brereton@leeds.ac.uk">A.M.Brereton@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/686/ruksana-bukhari-bibi">Bukhari-Bibi</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Ruksana</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6788</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.Bukhari-Bibi@leeds.ac.uk">R.Bukhari-Bibi@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Education Service Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=bulpitt-andy.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=bulpitt-andy.jpg&amp;jpegquality=80" alt="Professor Andy Bulpitt">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/115/professor-andy-bulpitt">Bulpitt</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Andy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6816</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.J.Bulpitt@leeds.ac.uk">A.J.Bulpitt@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11489/martin-callaghan">Callaghan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Martin</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 0273</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Callaghan@leeds.ac.uk">M.Callaghan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=carr-hamish.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=carr-hamish.jpg&amp;jpegquality=80" alt="Professor Hamish Carr">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/499/professor-hamish-carr">Carr</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hamish</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 7042</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.Carr@leeds.ac.uk">H.Carr@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=514ae3c1e3b40f87ab59a340c2ddd2b0.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=514ae3c1e3b40f87ab59a340c2ddd2b0.jpg&amp;jpegquality=80" alt="Dr Natasha Shakhlevich">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/285/dr-natasha-shakhlevich">Chakhlevitch</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Natasha Shakhlevich</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5444</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.Shakhlevich@leeds.ac.uk">N.Shakhlevich@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12765/omar-shaur-choudhry">Choudhry</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Omar</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:O.Choudhry@leeds.ac.uk">O.Choudhry@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=thumbnail_20220429_101925.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=thumbnail_20220429_101925.jpg&amp;jpegquality=80" alt="Deborah Clarke">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11490/deborah-clarke">Clarke</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Deborah</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.Clarke2@leeds.ac.uk">D.Clarke2@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Administrative Support Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e8c55345a06c2c397c31c5eabb95c8fa.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e8c55345a06c2c397c31c5eabb95c8fa.jpg&amp;jpegquality=80" alt="Professor Netta Cohen">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/301/professor-netta-cohen">Cohen</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Netta</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6789</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.Cohen@leeds.ac.uk">N.Cohen@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Tony_Cohn.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Tony_Cohn.jpg&amp;jpegquality=80" alt="Professor Anthony (Tony) G Cohn, FREng, CEng, CITP">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/76/professor-anthony-tony-g-cohn-freng-ceng-citp">Cohn</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Anthony</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5482</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.G.Cohn@leeds.ac.uk">A.G.Cohn@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Automated Reasoning</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_3.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_3.png&amp;jpegquality=80" alt="Mr. Minh Thang Dang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10328/mr-minh-thang-dang">Dang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Minh</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.T.Dang@leeds.ac.uk">M.T.Dang@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Marc_Dekamps.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Marc_Dekamps.jpg&amp;jpegquality=80" alt="Dr Marc de Kamps">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/390/dr-marc-de-kamps">de Kamps</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Marc</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5322</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.deKamps@leeds.ac.uk">M.deKamps@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8419/sara">Dickinson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sara</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 7049</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.D.J.Dickinson@leeds.ac.uk">S.D.J.Dickinson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Administrative Support Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Vania_Dimitrova.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Vania_Dimitrova.jpg&amp;jpegquality=80" alt="Professor Vania Dimitrova">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/184/professor-vania-dimitrova">Dimitrova</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Vania</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1674</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:V.G.Dimitrova@leeds.ac.uk">V.G.Dimitrova@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Human-Centred Artificial Intelligence</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=djemame-karim.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=djemame-karim.jpg&amp;jpegquality=80" alt="Professor Karim Djemame">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/187/professor-karim-djemame">Djemame</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Karim</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6590</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:K.Djemame@leeds.ac.uk">K.Djemame@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Mehmet_Dogar.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Mehmet_Dogar.jpg&amp;jpegquality=80" alt="Dr Mehmet Dogar">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/743/dr-mehmet-dogar">Dogar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mehmet</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5777</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.R.Dogar@leeds.ac.uk">M.R.Dogar@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Duke_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Duke_1.jpg&amp;jpegquality=80" alt="Professor David Duke">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/334/professor-david-duke">Duke</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">David</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6800</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.J.Duke@leeds.ac.uk">D.J.Duke@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor Emeritus of Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Martin_Dyer_2.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Martin_Dyer_2.jpg&amp;jpegquality=80" alt="Professor Martin Dyer">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/52/professor-martin-dyer">Dyer</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Martin</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5442</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.E.Dyer@leeds.ac.uk">M.E.Dyer@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=76098627e62aba735fb54d64cebf8235.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=76098627e62aba735fb54d64cebf8235.jpg&amp;jpegquality=80" alt="Dr Nick Efford">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/92/dr-nick-efford">Efford</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Nick</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6809</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.D.Efford@leeds.ac.uk">N.D.Efford@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__79__1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__79__1.png&amp;jpegquality=80" alt="Dr Samson Fabiyi">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9808/dr-samson-fabiyi">Fabiyi</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Samson</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2463</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.D.Fabiyi@leeds.ac.uk">S.D.Fabiyi@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Transnational Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8f6e158936e73a2a2f4639d2d2cae506.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8f6e158936e73a2a2f4639d2d2cae506.jpg&amp;jpegquality=80" alt="Charlotte Francis">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/321/charlotte-francis">Francis</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Charlotte</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5821</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:C.Francis@leeds.ac.uk">C.Francis@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Assistant School Education Service Mger</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FrangiMini2016SQ.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FrangiMini2016SQ.png&amp;jpegquality=80" alt="Professor Alejandro F Frangi">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1535/professor-alejandro-f-frangi">Frangi</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Alex</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 9640</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Frangi@leeds.ac.uk">A.Frangi@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Diamond Jubilee Chair in Computational Medicine | Royal Academy </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/6424/dr-helen-freeman">Freeman</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Helen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 9071</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.M.Freeman@leeds.ac.uk">H.M.Freeman@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Network Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Joseph.JPG&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Joseph.JPG&amp;jpegquality=80" alt="Joseph Ghaffari Motlagh">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8191/joseph-ghaffari-motlagh">Ghaffari Motlagh</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yousef</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3449</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.GhaffariMotlagh@leeds.ac.uk">Y.GhaffariMotlagh@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Visiting Academic</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8344/sunjeev-ghir">Ghir</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sunjeev</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1650</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Ghir@leeds.ac.uk">S.Ghir@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">TBC</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1837/shokoufeh-golshani">Golshani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Shokoufeh</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Golshani@leeds.ac.uk">S.Golshani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Marie Curie Researcher</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8d68a96aa6dc6bdea3056b634b2b7dd1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8d68a96aa6dc6bdea3056b634b2b7dd1.jpg&amp;jpegquality=80" alt="Anne Hayler">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/332/anne-hayler">Hayler</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Anne</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2228</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Hayler@leeds.ac.uk">A.Hayler@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">School Education Service Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=hazlehurst-thomas.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=hazlehurst-thomas.jpg&amp;jpegquality=80" alt="Dr Tom Hazlehurst">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/chemical-engineering/staff/835/dr-tom-hazlehurst">Hazlehurst</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thomas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Hazlehurst@leeds.ac.uk">T.Hazlehurst@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Head.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Head.jpg&amp;jpegquality=80" alt="Dr David Head">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/542/dr-david-head">Head</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">David</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4693</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.Head@leeds.ac.uk">D.Head@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/283/beth-hilditch">Hilditch</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Beth</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5465</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.A.Hilditch@leeds.ac.uk">B.A.Hilditch@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Education Service Officer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_3474_window.JPG&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_3474_window.JPG&amp;jpegquality=80" alt="Professor David Hogg">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/84/professor-david-hogg">Hogg</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">David</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5765</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.C.Hogg@leeds.ac.uk">D.C.Hogg@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Artificial Intelligence</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Kattia_4.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Kattia_4.png&amp;jpegquality=80" alt="Mrs. Kattia Lisette Hoz de Vila">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6749/mrs-kattia-lisette-hoz-de-vila">Hoz de Vila Eduardo</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Kattia</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:K.HozdeVila@leeds.ac.uk">K.HozdeVila@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Software Engineer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image823.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image823.png&amp;jpegquality=80" alt="Dr Yanlong Huang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8178/dr-yanlong-huang">Huang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yanlong</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3505</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.L.Huang@leeds.ac.uk">Y.L.Huang@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">University Academic Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jimack_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jimack_1.jpg&amp;jpegquality=80" alt="Professor Peter Jimack">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/82/professor-peter-jimack">Jimack</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Peter</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2002</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:P.K.Jimack@leeds.ac.uk">P.K.Jimack@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Scientific Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Owen_Johnson_2.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Owen_Johnson_2.jpg&amp;jpegquality=80" alt="Owen Johnson">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/259/owen-johnson">Johnson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Owen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5459</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:O.A.Johnson@leeds.ac.uk">O.A.Johnson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__94_.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__94_.png&amp;jpegquality=80" alt="Dr Rafael Kuffner dos Anjos">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9770/dr-rafael-kuffner-dos-anjos">Kuffner Dos Anjos</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Rafael</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3625</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.KuffnerdosAnjos@leeds.ac.uk">R.KuffnerdosAnjos@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computer Graphics</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=satish_2.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=satish_2.jpg&amp;jpegquality=80" alt="Dr Satish Kumar">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9634/dr-satish-kumar">Kumar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Satish</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3722</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Kumar3@leeds.ac.uk">S.Kumar3@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=d31c04b9b18f187adf52203ce1b97155.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=d31c04b9b18f187adf52203ce1b97155.jpg&amp;jpegquality=80" alt="Professor Raymond Kwan">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/37/professor-raymond-kwan">Kwan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Raymond</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5760</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.S.Kwan@leeds.ac.uk">R.S.Kwan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Scheduling</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=newphoto_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=newphoto_1.jpg&amp;jpegquality=80" alt="Dr Toni Lassila">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1542/dr-toni-lassila">Lassila</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Toni</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3724</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Lassila@leeds.ac.uk">T.Lassila@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lau3_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lau3_1.jpg&amp;jpegquality=80" alt="Kelvin Lau">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8195/kelvin-lau">Lau</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hui</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3748</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.K.Lau@leeds.ac.uk">H.K.Lau@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10007/dr-wiktoria-lawniczak">Lawniczak</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Wiktoria</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:W.Lawniczak@leeds.ac.uk">W.Lawniczak@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">KTP Associate – Senior Data Scientist</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ale_passport_uk___400X400.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ale_passport_uk___400X400.jpg&amp;jpegquality=80" alt="Mr. Alejandro Lebrero">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6758/mr-alejandro-lebrero">Lebrero</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Alejandro</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Lebrero@leeds.ac.uk">A.Lebrero@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Software Engineer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=leng-joanna.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=leng-joanna.jpg&amp;jpegquality=80" alt="Joanna Leng">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1509/joanna-leng">Leng</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Joanna</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3809</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.Leng@leeds.ac.uk">J.Leng@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Research Software Engineering Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('');">
                    
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/771/dr-matteo-leonetti">Leonetti</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Matteo</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5792</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Leonetti@leeds.ac.uk">M.Leonetti@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=JianLiu.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=JianLiu.jpg&amp;jpegquality=80" alt="Dr Jian Liu">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9216/dr-jian-liu">Liu</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Jian</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.Liu9@leeds.ac.uk">J.Liu9@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/11398/mr-lukasz-liwski">Liwski</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Lukasz</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:L.Liwski@leeds.ac.uk">L.Liwski@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Student Education Service Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Derek_Masee.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Derek_Masee.jpg&amp;jpegquality=80" alt="Derek Magee">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/172/derek-magee">Magee</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Derek</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6819</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.R.Magee@leeds.ac.uk">D.R.Magee@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=perfil_franco.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=perfil_franco.jpg&amp;jpegquality=80" alt="Franco Matzkin">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11493/franco-matzkin">Matzkin</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Victor</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:V.F.Matzkin@leeds.ac.uk">V.F.Matzkin@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">PhD Student</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Dr_Mantas_Mikaitis.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Dr_Mantas_Mikaitis.png&amp;jpegquality=80" alt="Dr Mantas Mikaitis">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12197/dr-mantas-mikaitis">Mikaitis</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mantas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Mikaitis@leeds.ac.uk">M.Mikaitis@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=muller-haiko.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=muller-haiko.jpg&amp;jpegquality=80" alt="Dr Haiko Muller">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/241/dr-haiko-muller">Muller</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Haiko</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5445</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.Muller@leeds.ac.uk">H.Muller@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11922/dr-zeinab-nezami">Nezami</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Zeinab</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Z.Nezami@leeds.ac.uk">Z.Nezami@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/chemical-engineering/staff/6598/dr-thai-thu-hien-nguyen">Nguyen</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hien</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.Nguyen1@leeds.ac.uk">H.Nguyen1@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8268/thanh-nguyen">Nguyen</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thanh-Luan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4315</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.L.Nguyen@leeds.ac.uk">T.L.Nguyen@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">TBC</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/electronic-engineering/staff/6843/dr-luzhen-nie">Nie</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Luzhen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:L.Nie@leeds.ac.uk">L.Nie@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Sebastian_ordyniak.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Sebastian_ordyniak.jpg&amp;jpegquality=80" alt="Dr. Sebastian Ordyniak">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8786/dr-sebastian-ordyniak">Ordyniak</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sebastian</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Ordyniak@leeds.ac.uk">S.Ordyniak@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer (Algorithms and Complexity)</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10329/dr-giacomo-paesani">Paesani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Giacomo</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:G.Paesani@leeds.ac.uk">G.Paesani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow (Algorithms and Complexity)</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_6610.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_6610.png&amp;jpegquality=80" alt="Dr Rafael Papallas">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6769/dr-rafael-papallas">Papallas</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Rafael</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.Papallas@leeds.ac.uk">R.Papallas@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=a27baf9e9e032626ba19c085393ce68c.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=a27baf9e9e032626ba19c085393ce68c.jpg&amp;jpegquality=80" alt="Dr Jonathan Pickering">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/540/dr-jonathan-pickering">Pickering</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Jonathan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5836</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.H.Pickering@leeds.ac.uk">J.H.Pickering@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo5_1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo5_1.png&amp;jpegquality=80" alt="Dr Evangelos Pournaras">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6446/dr-evangelos-pournaras">Pournaras</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Evangelos</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:E.Pournaras@leeds.ac.uk">E.Pournaras@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=44697006_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=44697006_1.jpg&amp;jpegquality=80" alt="Dr. Arash Rabbani">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11422/dr-arash-rabbani">Rabbani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Arash</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Rabbani@leeds.ac.uk">A.Rabbani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer (Assistant Professor)</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AL4_3530_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AL4_3530_1.jpg&amp;jpegquality=80" alt="Dr Thomas Ranner">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/643/dr-thomas-ranner">Ranner</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thomas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4697</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Ranner@leeds.ac.uk">T.Ranner@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Nishant_Ravikumar.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Nishant_Ravikumar.jpg&amp;jpegquality=80" alt="Dr Nishant Ravikumar">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1846/dr-nishant-ravikumar">Ravikumar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Nishant</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.Ravikumar@leeds.ac.uk">N.Ravikumar@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/maths/staff/12381/meg-richards">Richards</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Megan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.K.Richards@leeds.ac.uk">M.K.Richards@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Demonstrator/Module Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8787/dr-thomas-richardson">Richardson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thomas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Richardson1@leeds.ac.uk">T.Richardson1@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e9d4d4a1516f5a1ac703735f55fa7df7.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e9d4d4a1516f5a1ac703735f55fa7df7.jpg&amp;jpegquality=80" alt="Professor Roy Ruddle">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/257/professor-roy-ruddle">Ruddle</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Roy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1711</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.A.Ruddle@leeds.ac.uk">R.A.Ruddle@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9772/hamidreza-saligheh-rad">Saligheh Rad</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hamidreza</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.SalighehRad@leeds.ac.uk">H.SalighehRad@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">TBC</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lDzjBFOo_400x400.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lDzjBFOo_400x400.jpg&amp;jpegquality=80" alt="Dr. Duygu Sarikaya">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/13320/dr-duygu-sarikaya">Sarikaya</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Duygu</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.Sarikaya@leeds.ac.uk">D.Sarikaya@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=107_1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=107_1.png&amp;jpegquality=80" alt="Dr Ali Sarrami-Foroushani">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6780/dr-ali-sarrami-foroushani">Sarrami Foroushani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Ali</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Sarrami@leeds.ac.uk">A.Sarrami@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=2c0af244528dcf4bf817f6c7d9e3919d.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=2c0af244528dcf4bf817f6c7d9e3919d.jpg&amp;jpegquality=80" alt="Claire Savy">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/621/claire-savy">Savy</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Claire</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5449</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:C.Savy@leeds.ac.uk">C.Savy@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Centre Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Steve.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Steve.jpg&amp;jpegquality=80" alt="Professor Steve Scott">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/4601/professor-steve-scott">Scott</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Stephen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.K.Scott@leeds.ac.uk">S.K.Scott@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Head of School</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_20221231_201248_008__1_.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_20221231_201248_008__1_.jpg&amp;jpegquality=80" alt="Abhinav Sharma">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12771/abhinav-sharma">Sharma</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Abhinav</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Sharma1@leeds.ac.uk">A.Sharma1@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Software Engineer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6782/giulia-sindoni">Sindoni</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Giulia</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4821</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:G.Sindoni@leeds.ac.uk">G.Sindoni@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Demonstrator/Module Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=f4f95c22c21ebd9763c6e9be275aa6d1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=f4f95c22c21ebd9763c6e9be275aa6d1.jpg&amp;jpegquality=80" alt="Dr John Stell">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/258/dr-john-stell">Stell</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">John</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1076</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.G.Stell@leeds.ac.uk">J.G.Stell@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=kristina.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=kristina.jpg&amp;jpegquality=80" alt="Professor Kristina Vušković">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/249/professor-kristina-vuskovic">Vuskovic</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Kristina</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5443</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:K.Vuskovic@leeds.ac.uk">K.Vuskovic@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Algorithms and Combinatorics</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=46d6e703fb7c08526d97393060bc23da.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=46d6e703fb7c08526d97393060bc23da.jpg&amp;jpegquality=80" alt="Dr Mark Walkley">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/189/dr-mark-walkley">Walkley</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mark</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5684</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.A.Walkley@leeds.ac.uk">M.A.Walkley@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=He_Wang.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=He_Wang.jpg&amp;jpegquality=80" alt="Dr He Wang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/868/dr-he-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">He</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5767</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.E.Wang@leeds.ac.uk">H.E.Wang@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo2_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo2_1.jpg&amp;jpegquality=80" alt="Dr Yongxing Wang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/1600/dr-yongxing-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yongxing</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4874</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.Wang3@leeds.ac.uk">Y.Wang3@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=portrait_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=portrait_1.jpg&amp;jpegquality=80" alt="Professor Zheng Wang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6452/professor-zheng-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Zheng</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1077</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Z.Wang5@leeds.ac.uk">Z.Wang5@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Intelligent Software Technology</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11423/xiao-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Xiao</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:X.Wang16@leeds.ac.uk">X.Wang16@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=99b9545809e957a4a544ef9d2202a1bd.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=99b9545809e957a4a544ef9d2202a1bd.jpg&amp;jpegquality=80" alt="Dr Lijun Wei">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/760/dr-lijun-wei">Wei</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Lijun</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:L.J.Wei@leeds.ac.uk">L.J.Wei@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9416/adam-wilkinson">Wilkinson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Adam</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.J.Wilkinson@leeds.ac.uk">A.J.Wilkinson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Student Education Service Assistant Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=wilson-samuel.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=wilson-samuel.jpg&amp;jpegquality=80" alt="Dr Samuel Wilson">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/573/dr-samuel-wilson">Wilson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Samuel</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5474</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.S.Wilson@leeds.ac.uk">S.S.Wilson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer of Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8269/dr-xia">Xia</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.Xia@leeds.ac.uk">Y.Xia@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jie_Xu.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jie_Xu.jpg&amp;jpegquality=80" alt="Professor Jie Xu">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/331/professor-jie-xu">Xu</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Jie</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5193</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.Xu@leeds.ac.uk">J.Xu@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FB_IMG_1541828614755.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FB_IMG_1541828614755.jpg&amp;jpegquality=80" alt="Dr. Omer Yuval">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8789/dr-omer-yuval">Yuval</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Omer</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:O.Yuval@leeds.ac.uk">O.Yuval@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Arezoo_1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Arezoo_1.png&amp;jpegquality=80" alt="Dr. Arezoo Zakeri">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8199/dr-arezoo-zakeri">Zakeri</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Arezoo</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5254</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Zakeri@leeds.ac.uk">A.Zakeri@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                    </tbody>"""
url = []

soup = BeautifulSoup(html_my, 'html.parser')  # 创建BeautifulSoup对象

for link in soup.find_all('a'):
    href = link.get('href')
    if href.startswith('https'):  # 只输出以https开头的链接
        url.append(href)

In [24]:
print(url)

['https://eps.leeds.ac.uk/computing/staff/8807/dr-noorhan-abbas', 'https://eps.leeds.ac.uk/computing/staff/13233/professor-isolde-adler', 'https://eps.leeds.ac.uk/mechanical-engineering/staff/12626/dr-ban-al-jassani', 'https://eps.leeds.ac.uk/computing/staff/10225/dr-bogdan-alecu', 'https://eps.leeds.ac.uk/computing/staff/11465/dr-sharib-ali', 'https://eps.leeds.ac.uk/computing/staff/635/dr-mohammad-ammar-alsalka', 'https://eps.leeds.ac.uk/computing/staff/8784/dr-abdulrahman-altahhan', 'https://eps.leeds.ac.uk/computing/staff/33/professor-eric-atwell', 'https://eps.leeds.ac.uk/computing/staff/12946/dr-zeeshan', 'https://eps.leeds.ac.uk/computing/staff/10151/dr-fatemeh-banaie', 'https://eps.leeds.ac.uk/computing/staff/6763/dr-amy-lowe', 'https://eps.leeds.ac.uk/computing/staff/121/dr-brandon-bennett', 'https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/13223/harriet-bennett', 'https://eps.leeds.ac.uk/computing/staff/8810/dr-markus-billeter', 'https://eps.leeds.ac.uk/com

In [12]:
df = pd.DataFrame(columns=['name', 'text'])

count = 0
for i in url:
    response = requests.get(i)
    time.sleep(0.5)
    soup = BeautifulSoup(response.text, 'html.parser')
    cms = soup.find('div', class_='cms')
    profile = cms.text

    df.loc[count, 'text'] = profile
    count = count + 1

### Text cleaning

In [28]:
def capitalize_words(s):
    return s.title()

names = []

for i in url:
    name = i.rsplit('/', 1)[-1]  # Split the URL from the end ('/') and take the last part (-1)
    newName = name.replace('-', ' ')
    newName = newName.title()
    names.append(newName)  # Replace hyphens with spaces

Dr Noorhan Abbas
Professor Isolde Adler
Dr Ban Al Jassani
Dr Bogdan Alecu
Dr Sharib Ali
Dr Mohammad Ammar Alsalka
Dr Abdulrahman Altahhan
Professor Eric Atwell
Dr Zeeshan
Dr Fatemeh Banaie
Dr Amy Lowe
Dr Brandon Bennett
Harriet Bennett
Dr Markus Billeter
Dr Peter Bollada
Amy Brereton
Ruksana Bukhari Bibi
Professor Andy Bulpitt
Martin Callaghan
Professor Hamish Carr
Dr Natasha Shakhlevich
Omar Shaur Choudhry
Deborah Clarke
Professor Netta Cohen
Professor Anthony Tony G Cohn Freng Ceng Citp
Mr Minh Thang Dang
Dr Marc De Kamps
Sara
Professor Vania Dimitrova
Professor Karim Djemame
Dr Mehmet Dogar
Professor David Duke
Professor Martin Dyer
Dr Nick Efford
Dr Samson Fabiyi
Charlotte Francis
Professor Alejandro F Frangi
Dr Helen Freeman
Joseph Ghaffari Motlagh
Sunjeev Ghir
Shokoufeh Golshani
Anne Hayler
Dr Tom Hazlehurst
Dr David Head
Beth Hilditch
Professor David Hogg
Mrs Kattia Lisette Hoz De Vila
Dr Yanlong Huang
Professor Peter Jimack
Owen Johnson
Dr Rafael Kuffner Dos Anjos
Dr Satish Kum

In [31]:
df['name'] = pd.Series(names)

In [101]:
def expand_contractions(text):
    contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have","you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    
    def replace(match):
        return contractions_dict[match.group(0)]
    
    return contractions_re.sub(replace, text)

In [102]:
# 定义一个函数来清洁文本
def clean_text(text):
    # 移除HTML标签
    no_html = re.sub('<.*?>', '', text)
    # 移除 'Publications:' 和它之后的所有文本
    no_publications = re.sub('Publications:.*', '', no_html, flags=re.S)
    # 移除 '\xa0'
    no_special_char = re.sub('\xa0', ' ', no_publications)
    # 移除 '\n'
    cleaned_text = re.sub('\n', ' ', no_special_char)
    # 移除连续的空白字符
    cleaned_text = re.sub(' +', ' ', cleaned_text)

    return cleaned_text.strip()  # .strip() 移除字符串开头和结尾的空白字符

# 应用清洁函数到'text'列上
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].apply(expand_contractions)

In [104]:
df.to_csv('original_data.csv', index=False)

### Text filtering and further cleaning

In [179]:
# 加载spaCy英文模型
nlp = spacy.load('en_core_web_sm')

def filter_words(text):
    # 使用spaCy进行词性标注
    doc = nlp(text)
    
    # 只保留名词、形容词、副词和专有名词
    filtered_sentence = [token.text for token in doc if token.pos_ in ('NOUN', 'ADJ', 'ADV', 'PROPN')]
    
    return filtered_sentence

def clean_text1(text):
    to_remove = ['Research projects', 'Profile', 'Research groups and institutes', 
                 'Current postgraduate researchers', 'Postgraduate research opportunities', 
                 'Projects', 'Research interests', 'Research groups and institutes', 
                 'Research Funding', 'Qualifications', 'Student education', 'Student feedback', 
                 'Current MSc Researchers', 'Current postgraduate researchers', 
                 'Postgraduate research opportunities', 'Research Grants', 
                 'Responsibilities', 'Professional memberships', 'Current postgraduate researchers']

    for phrase in to_remove:
        text = text.replace(phrase, '')
    # 删除html标签
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 删除Publications: 之后的文字，包括publications 
    text = re.split(r'publications', text, flags=re.I)[0]

    # 删除\xa0和\n等非法字符
    text = text.replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ')

    # 删除额外的空格
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def clean_text2(text):
    to_remove = ['Research projects', 'Profile', 'Research groups and institutes', 
                 'Current postgraduate researchers', 'Postgraduate research opportunities', 
                 'Projects', 'Research interests', 'Research groups and institutes', 
                 'Research Funding', 'Qualifications', 'Student education', 'Student feedback', 
                 'Current MSc Researchers', 'Current postgraduate researchers', 
                 'Postgraduate research opportunities', 'Research Grants', 
                 'Responsibilities', 'Professional memberships', 'Current postgraduate researchers']

    for phrase in to_remove:
        text = text.replace(phrase, '')
    # 删除html标签
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 删除Publications: 之后的文字，包括publications 
    text = re.split(r'publications', text, flags=re.I)[0]

    # 删除\xa0和\n等非法字符
    text = text.replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ')

    # 删除额外的空格
    text = re.sub(r'\s+', ' ', text).strip()

    # 删除主语、动词和连接词
    text = ' '.join(filter_words(text))

    return text

### Self-trained Word2Vec model for recommendation

In [157]:
def recommend_tutors(df, user_input):
    # 清理文本并进行分词
    df['text_cleaned'] = df['text'].apply(lambda x: nltk.word_tokenize(clean_text1(x).lower()))

    # 训练Word2Vec模型
    model = Word2Vec(sentences=df['text_cleaned'], vector_size=100, window=5, min_count=1, workers=4)

    # 定义函数将文本转换为向量
    def text_to_vector(word_list):
        # 使用Word2Vec模型将每个单词转换为向量，然后取平均
        word_vectors = np.array([model.wv[word] for word in word_list if word in model.wv.index_to_key])
        return word_vectors.mean(axis=0) if word_vectors.size != 0 else np.zeros(model.vector_size)

    # 将导师描述和用户输入转换为向量
    df['vector'] = df['text_cleaned'].apply(text_to_vector)
    user_vector = text_to_vector(nltk.word_tokenize(clean_text2(user_input).lower())) # 删除主语和动词

    # 计算相似度并推荐导师
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([x], [user_vector])[0][0])
    recommended_tutors = df.sort_values('similarity', ascending=False)

    return recommended_tutors

input_text = "I like playing games and artificial intelligence"
recommended_tutors1 = recommend_tutors(df, input_text)

In [159]:
recommended_tutors1.head(5)

,name,text,text_cleaned,vector,similarity
83,Claire Savy,Responsibilities EPSRC Centre for Doctoral Tra...,"[epsrc, centre, for, doctoral, training, leeds...","[-0.21211824, 0.23655954, -0.052094925, 0.0533...",0.999653
7,Professor Eric Atwell,Profile I teach and research in Artificial Int...,"[i, teach, and, research, in, artificial, inte...","[-0.23176491, 0.26142123, -0.058098283, 0.0579...",0.999647
47,Dr Yanlong Huang,Profile I am a University Academic Fellow at t...,"[i, am, a, university, academic, fellow, at, t...","[-0.23331219, 0.2620863, -0.059442654, 0.05649...",0.999521
98,Professor Jie Xu,Profile I am the leader for a Research Peak of...,"[i, am, the, leader, for, a, research, peak, o...","[-0.26310083, 0.29256654, -0.0677047, 0.064671...",0.999477
65,Dr Zeinab Nezami,Profile I have joined the School of Computing ...,"[i, have, joined, the, school, of, computing, ...","[-0.2554071, 0.28333068, -0.065902814, 0.06214...",0.999459


### Google word2vec model: GoogleNews-vectors-negative300

In [160]:
model_name = "word2vec-google-news-300"  # 模型名称
model_fasttext = api.load(model_name)

In [166]:
# 加载Google News Word2Vec模型
model = model_fasttext

def text_to_vector(word_list):
    # 使用Word2Vec模型将每个单词转换为向量，然后取平均
    word_vectors = np.array([model[word] for word in word_list if word in model])
    return word_vectors.mean(axis=0) if word_vectors.size != 0 else np.zeros(300)

def recommend_tutors(df, user_input):
    # 清理文本并进行分词
    df['text_cleaned'] = df['text'].apply(lambda x: nltk.word_tokenize(clean_text1(x).lower()))

    # 将导师描述和用户输入转换为向量
    df['vector'] = df['text_cleaned'].apply(text_to_vector)
    user_vector = text_to_vector(nltk.word_tokenize(clean_text2(user_input).lower()))
  
    # 计算相似度并推荐导师
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([x], [user_vector])[0][0])
    recommended_tutors = df.sort_values('similarity', ascending=False)

    return recommended_tutors

input_text = "I like softerware engineering"
recommended_tutors2 = recommend_tutors(df, input_text)

In [167]:
recommended_tutors2.head(5)

,name,text,text_cleaned,vector,similarity
2,Dr Ban Al Jassani,Profile Senior programmer with valuable experi...,"[senior, programmer, with, valuable, experienc...","[-0.011360963, -0.01838843, 0.02000157, 0.1091...",0.567456
78,Dr Thomas Richardson,Research interests Real-time systems Safety-cr...,"[real-time, systems, safety-critical, systems,...","[-0.008566463, -0.015745275, 0.07062486, 0.136...",0.560547
89,Dr Mark Walkley,Profile I began by studying for a BSc Engineer...,"[i, began, by, studying, for, a, bsc, engineer...","[-0.010910034, 0.008874767, 0.06359244, 0.1172...",0.546399
66,Dr Thai Thu Hien Nguyen,"Profile 2007 – 2008 MSc Chemical Engineering, ...","[2007, –, 2008, msc, chemical, engineering, ,,...","[-0.07033644, 0.0065768375, 0.061653886, 0.093...",0.545763
72,Dr Jonathan Pickering,Profile Research Metallurgist at what is now p...,"[research, metallurgist, at, what, is, now, pa...","[-0.008868448, -0.013957989, 0.03588578, 0.065...",0.507472


### jaccard similarity

In [177]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

def recommend_tutors(df, user_input):
    # 清理文本并进行分词
    df['words_set'] = df['text'].apply(lambda x: set(nltk.word_tokenize(clean_text1(x).lower())))

    # 将用户输入转换为词集合
    user_words_set = set(nltk.word_tokenize(clean_text1(user_input).lower()))

    # 计算Jaccard相似度并推荐导师
    df['similarity'] = df['words_set'].apply(lambda x: jaccard_similarity(x, user_words_set))
    recommended_tutors = df.sort_values('similarity', ascending=False)

    return recommended_tutors

input_text = "I like data mining and text analysis, and I am good at coding and I am also interested in artificial intelligence"
recommended_tutors3 = recommend_tutors(df, input_text)

In [178]:
recommended_tutors3.head(5)

,name,text,text_cleaned,vector,similarity,words_set
71,Dr Rafael Papallas,Profile I am a Research Fellow in the School o...,"[i, am, a, research, fellow, in, the, school, ...","[0.015335693, 0.023737794, -0.0051464844, 0.07...",0.171429,"{the, ,, planning, ., control, interest, i, a,..."
94,Dr Lijun Wei,Research projects Any research projects I am c...,"[any, research, projects, i, am, currently, wo...","[-0.032543946, 0.02414856, 0.03423055, 0.10489...",0.150000,"{currently, the, allows, be, full, ., any, sea..."
6,Dr Abdulrahman Altahhan,Profile I have extesive experience in developi...,"[i, have, extesive, experience, in, developing...","[-0.032901, 0.0530971, 0.034158763, 0.08209403...",0.130435,"{applications, leading, the, reinforcement, ,,..."
5,Dr Mohammad Ammar Alsalka,Responsibilities MSc Projects Coordinator Rese...,"[msc, coordinator, natural, language, processi...","[-0.020472085, 0.014969175, 0.031466227, 0.078...",0.106383,"{world, the, saud, around, are, alshammari, fu..."
17,Professor Andy Bulpitt,Responsibilities Head of School Research inter...,"[head, of, school, andy, is, research, interes...","[-0.030078987, 0.031551972, 0.043749295, 0.098...",0.105882,"{world, the, are, :, dphil, around, full, ., a..."


Doc2Vec for input and 

In [189]:
def tag_docs(docs):
    tagged = [TaggedDocument(words=words, tags=[i]) for i, words in enumerate(docs)]
    return tagged

def train_doc2vec_model(tagged_docs):
    model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4)
    model.build_vocab(tagged_docs)
    model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)
    return model

def vectorize_docs(model, tagged_docs):
    vectors = [model.infer_vector(doc.words, epochs=50) for doc in tagged_docs]
    return vectors

# 清理文本并进行分词
df['text_cleaned'] = df['text'].apply(lambda x: nltk.word_tokenize(clean_text1(x).lower()))

input_text = "I like data mining and text analysis"

# 将用户输入也转换为清理后的词列表
user_input = nltk.word_tokenize(clean_text1(input_text).lower())

# 为导师描述和用户输入生成TaggedDocument对象
tagged_tutor_descs = tag_docs(df['text_cleaned'])
tagged_user_input = TaggedDocument(words=user_input, tags=['user_input'])

# 训练Doc2Vec模型
model = train_doc2vec_model(tagged_tutor_descs)

# 将导师描述和用户输入转换为向量
df['vector'] = vectorize_docs(model, tagged_tutor_descs)
user_vector = model.infer_vector(user_input, epochs=50)

# 计算相似度并推荐导师
df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([x], [user_vector])[0][0])
recommended_tutors4 = df.sort_values('similarity', ascending=False)

In [190]:
recommended_tutors4

,name,text,text_cleaned,vector,similarity,words_set,text_cleaned_string
6,Dr Abdulrahman Altahhan,Profile I have extesive experience in developi...,"[i, have, extesive, experience, in, developing...","[-0.4193915, 0.23988488, -0.38301352, 0.446977...",0.994839,"{applications, leading, the, reinforcement, ,,...","i have extesive experience in developing , lea..."
56,Mr Alejandro Lebrero,Profile Alejandro Lebrero is a Research Softwa...,"[alejandro, lebrero, is, a, research, software...","[-1.1530976, 0.54556406, -0.912837, 1.0264353,...",0.991548,"{through, others, the, specialists, fields, ap...",alejandro lebrero is a research software engin...
47,Dr Yanlong Huang,Profile I am a University Academic Fellow at t...,"[i, am, a, university, academic, fellow, at, t...","[-1.0480638, 0.6380577, -0.9759443, 1.2768623,...",0.988610,"{null, the, shared, are, :, legged, physical, ...",i am a university academic fellow at the schoo...
66,Dr Thai Thu Hien Nguyen,"Profile 2007 – 2008 MSc Chemical Engineering, ...","[2007, –, 2008, msc, chemical, engineering, ,,...","[-0.60022116, 0.30730405, -0.5461798, 0.616279...",0.985783,"{currently, the, allows, ,, –, 2008, be, full,...","2007 – 2008 msc chemical engineering , univers..."
93,Xiao Wang,Research projects Any research projects I am c...,"[any, research, projects, i, am, currently, wo...","[-0.31992438, 0.19313335, -0.30312338, 0.34853...",0.984128,"{currently, the, allows, be, full, ., any, sea...",any research projects i am currently working o...
...,...,...,...,...,...,...,...
12,Harriet Bennett,,[],"[-0.0048435125, -0.0044348915, 0.0020702952, 0...",-0.061662,{},
60,Mr Lukasz Liwski,,[],"[0.00165447, -0.0023070527, 0.0021383197, 0.00...",-0.111344,{},
95,Adam Wilkinson,,[],"[0.00038955986, 0.0042228033, 0.0020591002, -0...",-0.114789,{},
35,Charlotte Francis,,[],"[-0.0026665784, -0.0033410636, 0.004266519, -0...",-0.130139,{},


### CountVectorizer

In [196]:
def recommend_tutors(df, user_input):
    # 清理文本
    df['text_cleaned'] = df['text'].apply(clean_text2) # 删除主语动词连接词，因为CountVectorizer只考虑词频，所以主语等会有很大影响
    
    # 使用CountVectorizer将导师描述转换为向量
    vectorizer = CountVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(df['text_cleaned'])
    
    # 使用相同的Vectorizer将用户输入转换为向量
    user_vector = vectorizer.transform([clean_text2(user_input)])
    
    # 计算相似度并推荐导师
    df['similarity'] = cosine_similarity(vectors, user_vector).flatten()
    recommended_tutors = df.sort_values('similarity', ascending=False)
    
    return recommended_tutors

input_text = "I like software engineering"
recommended_tutors5 = recommend_tutors(df, input_text)

In [197]:
recommended_tutors5

,name,text,text_cleaned,vector,similarity,words_set,text_cleaned_string
72,Dr Jonathan Pickering,Profile Research Metallurgist at what is now p...,Research Metallurgist now part Retrained Compu...,"[-1.2407446, 0.33904517, -1.0456468, 1.0950773...",0.418121,"{skills, ccdc, the, fields, :, application, ai...",research metallurgist at what is now part qine...
56,Mr Alejandro Lebrero,Profile Alejandro Lebrero is a Research Softwa...,Alejandro Lebrero Research Software Engineer C...,"[-1.1530976, 0.54556406, -0.912837, 1.0264353,...",0.390360,"{through, others, the, specialists, fields, ap...",alejandro lebrero is a research software engin...
89,Dr Mark Walkley,Profile I began by studying for a BSc Engineer...,BSc Engineering Mathematics Civil Engineering ...,"[-0.78988457, 0.2292327, -0.7243725, 0.8288290...",0.314270,"{world, the, supervised, are, around, studies,...",i began by studying for a bsc engineering math...
78,Dr Thomas Richardson,Research interests Real-time systems Safety-cr...,Real time systems Safety critical systems Embe...,"[-0.59115875, 0.19056779, -0.5674481, 0.628624...",0.312348,"{currently, the, allows, pgce, operating, arch...",real-time systems safety-critical systems embe...
66,Dr Thai Thu Hien Nguyen,"Profile 2007 – 2008 MSc Chemical Engineering, ...",MSc Chemical Engineering University Leeds UK P...,"[-0.60022116, 0.30730405, -0.5461798, 0.616279...",0.265165,"{currently, the, allows, ,, –, 2008, be, full,...","2007 – 2008 msc chemical engineering , univers..."
...,...,...,...,...,...,...,...
39,Sunjeev Ghir,,,"[0.004257243, -0.0010864257, 0.002064821, -0.0...",0.000000,{},
38,Joseph Ghaffari Motlagh,Research projects Any research projects I am c...,research projects currently below list researc...,"[-0.28913435, 0.11459136, -0.2890175, 0.314601...",0.000000,"{currently, the, allows, be, full, ., any, sea...",any research projects i am currently working o...
35,Charlotte Francis,,,"[-0.0026665784, -0.0033410636, 0.004266519, -0...",0.000000,{},
34,Dr Samson Fabiyi,Research projects Any research projects I am c...,research projects currently below list researc...,"[-0.42592597, 0.11235624, -0.3713601, 0.383613...",0.000000,"{currently, the, allows, be, full, ., any, sea...",any research projects i am currently working o...


### TF-IDF

In [206]:
def recommend_tutors(df, user_input):
    # 清理文本
    df['text_cleaned'] = df['text'].apply(clean_text1)
    
    # 使用TfidfVectorizer将导师描述转换为向量
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(df['text_cleaned'])
    
    # 使用相同的Vectorizer将用户输入转换为向量
    user_vector = vectorizer.transform([clean_text2(user_input)])
    
    # 计算相似度并推荐导师
    df['similarity'] = cosine_similarity(vectors, user_vector).flatten()
    recommended_tutors = df.sort_values('similarity', ascending=False)
    
    return recommended_tutors

input_text = "I like software engineering"
recommended_tutors6 = recommend_tutors(df, input_text)

In [207]:
recommended_tutors6

,name,text,text_cleaned,vector,similarity,words_set,text_cleaned_string
72,Dr Jonathan Pickering,Profile Research Metallurgist at what is now p...,Research Metallurgist at what is now part Qine...,"[-1.2407446, 0.33904517, -1.0456468, 1.0950773...",0.357098,"{skills, ccdc, the, fields, :, application, ai...",research metallurgist at what is now part qine...
56,Mr Alejandro Lebrero,Profile Alejandro Lebrero is a Research Softwa...,Alejandro Lebrero is a Research Software Engin...,"[-1.1530976, 0.54556406, -0.912837, 1.0264353,...",0.355159,"{through, others, the, specialists, fields, ap...",alejandro lebrero is a research software engin...
78,Dr Thomas Richardson,Research interests Real-time systems Safety-cr...,Real-time systems Safety-critical systems Embe...,"[-0.59115875, 0.19056779, -0.5674481, 0.628624...",0.281978,"{currently, the, allows, pgce, operating, arch...",real-time systems safety-critical systems embe...
89,Dr Mark Walkley,Profile I began by studying for a BSc Engineer...,I began by studying for a BSc Engineering Math...,"[-0.78988457, 0.2292327, -0.7243725, 0.8288290...",0.209070,"{world, the, supervised, are, around, studies,...",i began by studying for a bsc engineering math...
10,Dr Amy Lowe,Research projects Any research projects I am c...,Any research projects I am currently working o...,"[-0.49034914, 0.08380247, -0.44313312, 0.46998...",0.207806,"{comp2913, web, currently, the, allows, also, ...",any research projects i am currently working o...
...,...,...,...,...,...,...,...
39,Sunjeev Ghir,,,"[0.004257243, -0.0010864257, 0.002064821, -0.0...",0.000000,{},
38,Joseph Ghaffari Motlagh,Research projects Any research projects I am c...,Any research projects I am currently working o...,"[-0.28913435, 0.11459136, -0.2890175, 0.314601...",0.000000,"{currently, the, allows, be, full, ., any, sea...",any research projects i am currently working o...
35,Charlotte Francis,,,"[-0.0026665784, -0.0033410636, 0.004266519, -0...",0.000000,{},
34,Dr Samson Fabiyi,Research projects Any research projects I am c...,Any research projects I am currently working o...,"[-0.42592597, 0.11235624, -0.3713601, 0.383613...",0.000000,"{currently, the, allows, be, full, ., any, sea...",any research projects i am currently working o...


#### OLD

In [47]:
def preprocess_text(text):
    """
    对文本进行预处理
    """
    text = text.lower()  # 转换为小写字母
    text = text.replace("'", "")  # 去除单引号
    text = text.replace("-", "")  # 去除破折号
    text = "".join([char for char in text if char.isalpha() or char.isspace()])  # 保留字母和空格
    return text

def tokenize_text(text):
    """
    将文本转换为tokens列表
    """
    tokens = text.split()
    return tokens

def plot_most_frequent_words(tokens, title):
    """
    绘制文本中出现频率最高的词汇柱状图
    """
    fdist = FreqDist(tokens)
    words_df = pd.DataFrame({'word': list(fdist.keys()), 'count': list(fdist.values())})
    # 选择出现频率最高的前20个词汇
    d = words_df.nlargest(columns="count", n=10)
    plt.figure(figsize=(16, 9))
    sns.barplot(data=d, x="word", y="count")
    plt.title(title)
    plt.show()


def plot_pearson_correlation_coefficient(X, data):
    """
    绘制各个文本之间的Pearson相关系数热力图
    """
    corr_matrix = np.zeros((len(data), len(data)))
    for i, j in itertools.combinations(range(len(data)), 2):
        corr, _ = pearsonr(X.toarray()[i], X.toarray()[j])
        corr_matrix[i, j] = corr
        corr_matrix[j, i] = corr
    tick_labels = data['title']
    sns.set(style="white")
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    f, ax = plt.subplots(figsize=(10, 8))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5},
                xticklabels=tick_labels, yticklabels=tick_labels,
                annot=True, fmt='.2f', annot_kws={'size': 10})
    plt.title("Pearson Correlation Coefficient Matrix")
    plt.show()


def recommend_courses(sinput, df):
    # 引入CountVectorizer库，用于文本向量化
    vectorizer = CountVectorizer(tokenizer=tokenize_text, preprocessor=preprocess_text)

    # 将data中的文本转换成向量X，每行代表一个课程的向量表示
    X = vectorizer.fit_transform(df['text'])

    # 将用户输入的文本转换成向量表示
    student_input = vectorizer.transform(sinput.split())

    # 计算用户输入的文本和所有课程向量之间的余弦相似度
    cosine_sim = cosine_similarity(X, student_input)

    # 为每个课程在data中增加一列'similarity'，表示该课程与用户输入的文本的相似度得分
    df['similarity'] = cosine_sim.sum(1)

    # 根据'similarity'得分进行降序排列，选择前3个得分最高
    top_courses = df.sort_values('similarity', ascending=False).head(3)['Name']
    df = df.sort_values('similarity', ascending=False)
    #return top_courses.values
    return df

In [48]:
df['text'] = df['text'].apply(lambda x: re.sub(r'<.*?>', '', x)) # Remove HTML tags
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) # Remove punctuation
df['text'] = df['text'].apply(lambda x: x.lower()) # Convert to lowercase
# Tokenize data
nltk.download('punkt')
df['tokens'] = df['text'].apply(lambda x: nltk.word_tokenize(x))

# Remove stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Lemmatize data
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\田见萌\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\田见萌\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\田见萌\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
recommend_courses('I like deep learning', df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.444047,i am actively researching in the areas of deep...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.385935,i obtained my phd in image analysis awarded in...
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.368204,i teach and research in artificial intelligenc...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.264764,profile profile profileprofile profileprofilep...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.251773,profileescriptionpreviously a research softwar...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.230658,profilevertisementi am currently a research fe...
6,Zeeshan,\nprofile\ni have earned a phd degree in robot...,"[profile, earned, phd, degree, robotics, auton...",0.225647,hegemonyprofileescriptioni have earned a phd d...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.174870,i teach and research in artificial intelligenc...
9,Andy Bulpitt,\nresponsibilities\n\nhead of school\n\nresear...,"[responsibility, head, school, research, inter...",0.055132,suddenlyresponsibilitiespotionpotionpotionshie...
8,Peter Bollada,\nprofile\napplied maths and physics degree op...,"[profile, applied, math, physic, degree, open,...",0.027677,i am part of the modelling contingent of the l...


In [35]:
recommend_courses('I like playing piano', df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.334731,i teach and research in artificial intelligenc...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.323688,i obtained my phd in image analysis awarded in...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.230658,profilevertisementi am currently a research fe...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.226941,profile profile profileprofile profileprofilep...
6,Zeeshan,\nprofile\ni have earned a phd degree in robot...,"[profile, earned, phd, degree, robotics, auton...",0.180517,hegemonyprofileescriptioni have earned a phd d...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.142430,i am actively researching in the areas of deep...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.112096,i teach and research in artificial intelligenc...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.107903,profileescriptionpreviously a research softwar...
8,Peter Bollada,\nprofile\napplied maths and physics degree op...,"[profile, applied, math, physic, degree, open,...",0.027677,i am part of the modelling contingent of the l...
1,Ban Al-Jassani,\nprofile\nsenior programmer with valuable exp...,"[profile, senior, programmer, valuable, experi...",0.000000,corridorprofileescriptionsenior programmer wit...


In [36]:
recommend_courses("I like data mining and text analysis", df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.937247,i teach and research in artificial intelligenc...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.907763,profile profile profileprofile profileprofilep...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.892287,i teach and research in artificial intelligenc...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.846568,i obtained my phd in image analysis awarded in...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.661882,i am actively researching in the areas of deep...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.611448,profileescriptionpreviously a research softwar...
3,Mohammad Ammar Alsalka,\nresponsibilities\n\nmsc projects coordinator...,"[responsibility, msc, project, coordinator, re...",0.593944,suddenlyresponsibilitiesacebookacebookacebooks...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.593120,profilevertisementi am currently a research fe...
12,Natasha Shakhlevich,\nresponsibilities\n\nexaminations officer\n\n...,"[responsibility, examination, officer, researc...",0.474342,heresponsibilities are in the area of determin...
11,Hamish Carr,\n \n \n \n ...,"[research, project, research, project, im, cur...",0.424264,will Â d ________________________________...


In [53]:
recommend_courses("text analytics, data mining", df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.273515,i teach and research in artificial intelligenc...
3,Mohammad Ammar Alsalka,\nresponsibilities\n\nmsc projects coordinator...,"[responsibility, msc, project, coordinator, re...",0.269975,suddenlyresponsibilitiesacebookacebookacebooks...
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.267785,i teach and research in artificial intelligenc...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.113470,profile profile profileprofile profileprofilep...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.108917,i am actively researching in the areas of deep...
11,Hamish Carr,\n \n \n \n ...,"[research, project, research, project, im, cur...",0.070711,will Â d ________________________________...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.035968,profileescriptionpreviously a research softwar...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.032951,profilevertisementi am currently a research fe...
1,Ban Al-Jassani,\nprofile\nsenior programmer with valuable exp...,"[profile, senior, programmer, valuable, experi...",0.000000,corridorprofileescriptionsenior programmer wit...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.000000,i obtained my phd in image analysis awarded in...



Supervisors which have more contents will get more recommendations.  

Supervisors with short contents have a limited number of keywords and do not easily get recommendations.  

There are some supervisors who often get to be at the top of the recommendation table because their contents are so long and therefore the number of keywords is also bigger than others, and no matter what you enter, they always take the top spot in the recommendations.  

(Sharib Ali and Noorhan Abbas always gets high similarity scores.)

Text generation using Bart model

In [37]:
device = torch.device("mps:0")

In [38]:
# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
modelB = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Function to generate summaries
def generate_summary_brat(text):
    inputs = tokenizer.encode_plus(text, truncation=True, max_length=1024, return_tensors='pt')
    input_ids = inputs['input_ids'].to(modelB.device)
    attention_mask = inputs['attention_mask'].to(modelB.device)
    summary_ids = modelB.generate(input_ids, attention_mask=attention_mask, num_beams=8, max_length=100, early_stopping=False) # max_length=100 is original set
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [39]:
df['summary'] = df['text'].apply(generate_summary_brat)

In [40]:
df

,Name,text,tokens,similarity,summary
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.937247,i teach and research in artificial intelligenc...
1,Ban Al-Jassani,\nprofile\nsenior programmer with valuable exp...,"[profile, senior, programmer, valuable, experi...",0.293119,corridorprofileescriptionsenior programmer wit...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.846568,i obtained my phd in image analysis awarded in...
3,Mohammad Ammar Alsalka,\nresponsibilities\n\nmsc projects coordinator...,"[responsibility, msc, project, coordinator, re...",0.593944,suddenlyresponsibilitiesacebookacebookacebooks...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.661882,i am actively researching in the areas of deep...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.892287,i teach and research in artificial intelligenc...
6,Zeeshan,\nprofile\ni have earned a phd degree in robot...,"[profile, earned, phd, degree, robotics, auton...",0.406164,hegemonyprofileescriptioni have earned a phd d...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.593120,profilevertisementi am currently a research fe...
8,Peter Bollada,\nprofile\napplied maths and physics degree op...,"[profile, applied, math, physic, degree, open,...",0.290604,i am part of the modelling contingent of the l...
9,Andy Bulpitt,\nresponsibilities\n\nhead of school\n\nresear...,"[responsibility, head, school, research, inter...",0.385922,suddenlyresponsibilitiespotionpotionpotionshie...


In [41]:
df.loc[0, 'summary']

'i teach and research in artificial intelligence python for data science data mining and text analytics in the school of computing at university of leeds\xa0 i\xa0worked in the edubots project which is an erasmus knowledge alliance project that aims to\xa0improve results and raise\xa0attainment levels\xa0in european higher education institutions this\xa0project focussed on exploring best practices and innovative use of chatbots and creating a learning community of educators\xa0i have experience in organising conferences and workshops\xa0'

TextRank generation

In [169]:
# Dr Abdulrahman Altahhan

x ="""I have extesive experience in developing, leading and managing postgraduate and undergraduate computer science HE programmes, particularly in the areas of AI and Data Science.

I am actively researching in the areas of Deep Reinforcement Learning, Neural Network and Machine Learning with hands-on applications and publications in image processing, autonomous vehicles and robotics. Autonomous learning from data is second nature to me; particularly I enjoy developing reinforcement learning algorithms, several of which have both a highly practical and theoretical potential.

I have developed a tool that was adopted throughout a university for quantifying and accurately modeling the data from modules and course outcomes, these algorithms and research have a high potential for commercialisation.

I have led a small research group in AI and Robotics along with a few programmes in Data Science, Intelligent Systems and Information Technology for both the undergraduate and graduate levels. I have taught numerous modules, the majority of which in AI and Machine Learning.

My main research area is Deep Reinforcement Learning applied to Intelligent Agents. I have developed several novel RL and DRL algorithms that seamlessly integrate experience replay within the learning process of the agent. I have applied these on games, stochastic processes and on predicting control signals for a prosthetic limp. I have developed several robot navigation models that employed deep reinforcement learning to visually allow a robot to learn to navigate in an unknown environment without prior experience.

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Student education
I have developed material for several modules for the new MSc. in Artificial Intelligence ODL programme. I lead modules such as Data Science and Reinforcement Learning on Robotics. I have also led modules such as Machine Learning, Neural Networks and Big Data Analysis.

I have served as a Programme Director for MSc. Data Science and Computational Intelligence, where I enjoy setting up suitable strategies for this programme to be closely related to both the industry and research (my favourite balance), nurturing it into popularity and growth.

I have obtained managerial skills by undertaking different leading roles and responsibilities, ex. appointed as acting dean of my college and coordinating BSc. in Business Information Technology program for three years. 

My experience of educating professional students includes proficiency in teaching several AI and computer science topics in multicultural settings. Extensively proposed and championed several curriculums in Data Science and AI at the Master and Undergraduate levels. 

Postgraduate research opportunities
We welcome enquiries from motivated and qualified applicants from all around the world who are interested in PhD study. Our research opportunities allow you to search for projects and scholarships."""

def generate_summary_textRank(x):
    model = Word2Vec(
    sentences= [word_tokenize(x)],
        vector_size=100, window=5, min_count=5, workers=2
    )

    sentences = sent_tokenize(x)
    sentences_feat = [
        np.array([model.wv[x] for x in word_tokenize(sentence.lower()) if x in model.wv]).mean(0)
        for sentence in sentences]

    sentences_feat = np.array(sentences_feat)

    from sklearn.metrics.pairwise import cosine_similarity
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentences_feat[i].reshape(1,100), sentences_feat[j].reshape(1,100))[0,0]
                

    nx_graph = nx.from_numpy_array(sim_mat)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            nx_graph[2][3]['weight'] = sim_mat[i][j]
    scores = nx.pagerank(nx_graph)

    return sentences[np.argmax(list(scores.values()))]

In [43]:
generate_summary_textRank(x)

'I have led a small research group in AI and Robotics along with a few programmes in Data Science, Intelligent Systems and Information Technology for both the undergraduate and graduate levels.'

Comparison with Bart

In [44]:
generate_summary_brat(x)

'I have extesive experience in developing, leading and managing postgraduate and undergraduate computer science HE programmes, particularly in the areas of AI and Data Science. I have also led a small research group in AI and Robotics. I am actively researching in the area of Deep Reinforcement Learning, Neural Network and Machine Learning with hands-on applications and publications in image processing, autonomous vehicles and robotics. Autonomous learning from data is second nature to me; particularly I enjoy developing reinforcement learning algorithms,'

In [170]:
# Omar Shaur Choudhry

y = """Profile
Third-year computer science undergraduate student and teaching assistant at the University of Leeds. I am the founder and director of my own company OC Solutions Ltd, providing bespoke information technology and software solutions and consultancy.

Responsibilities
Undergraduate Teaching Assistant
Research interests
I have completed my dissertation on Survival Analysis with Neural Networks for Chronic Heart Failure Patients.

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Professional memberships
Student Membership - BCS, The Chartered Institute for IT
Student Membership - IET, Institution of Engineering and Technology
Student education
In Semester 1, I mainly aided Level 1 students in lab sessions, focusing on the Introduction to Programming and Procedural Programming modules. I also helped Level 2 students in Web Application Development and Level 3 students in User Interfaces, Machine Learning, and Computer Graphics.

In Semester 2, I mainly assisting Level 2 students with the following modules: Data Mining, Networks, Artificial Intelligence, and Compilers Design & Construction. I also helped Level 1 students in Introduction to Web Technologies, Object-Oriented Programming, and Programming Project, and Level 3 students in Web Services & Web Data and Intelligent Systems and Robotics.""" #10/07

In [171]:
z = """Profile
Senior programmer with valuable experience of MATLAB, Python, C++, Visual Basic and Visual C++ expertise. Development and implementation of several algorithms such as Human-Computer Interactions and optimisation of digital circuits.

Responsibilities
Research Fellow - School of Mechanical Engineering
Teaching Assistant - School of Computing
Research interests
Extensions to the Growth Method for the Design of Space-Frame Structures

Computer Aided Synthesis and Optimisation for Electronic Logic Circuits.

Human-Computer Interactive system for SCADA system

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Qualifications
PhD - Computer and Electronic Engineering - Edinburgh Napier University
MSc - Computer Engineering - University of Technology - IRAQ
BSc - Computer Engineering - University of Baghdad - IRAQ"""

In [172]:
t = """Profile
I obtained my PhD in image analysis (awarded in 01/2016) from the University of Lorraine from CNRS - CRAN laboratory at Nancy, France, where I was involved in the development of robust computer vision algorithms for monitoring bladder cancer progression by accurately creating mosaics of endoscopic videos. The key element in this work was to enable computer to match reliable features based on frame motion and perform mosaicking of the surface under observation (Thesis here). I did my master’s by research in Computer Vision from University of Burgundy, France (2010-2012). My master’s thesis was on “Retinal image analysis from fundus imaging”, which was a collaborative project between Oak Ridge National Laboratory, USA and Le2I, Le Creusot at the University of Bourgogne.

Prior to joining Leeds as University lecturer, I worked as a post-doctoral research associate at the Department of Engineering Sciencee, University of Oxford. I currently hold a visiting fellow position at the University of Oxford. During my post-doc term there, I designed robust models for computer assisted endoscopy in gastroenterology, especially the oesophagus. The work resulted in several high-quality publication and two patents (filed/submitted). The project was a part of NIHR Oxford biomedical research centre imaging theme on Computer aided endoscopy for Barrett’s oesophagus, for which I worked in close collaboration with consultant gastroenterologists from the Oxford NHS University Hospitals, At Oxford, I am currently co-supervising two DPhil students.

Previously, I also did a post doc at Biomedical and Computer vision group of Prof. Karl Rohr at the University of Heidelberg and DKFZ, Germany. During my work there I collaborated extensively with neuroscience researchers (led by Prof. Katrin Amunts) and physicists (led by Markus Axer) at Forschungszentrum, Jülich, Germany. I designed mathematically plausible phyics-based deformable multi-modal image registraiton techinque that allowed for precise 3D reconstruction of ultra high resolution (1.3um) 2D histology brain images between CCD acquired image and high-throughput polarized light microscopy (more info here).

I have worked as researcher in different capacities in leading research teams in France, Spain, Germany, Switzerland and the UK. I am an initiator and lead organiser of series of endoscopic computer vision challenge that started in 2019 (follow-us here).  Together with Prof. Adrien Bartoli and colleagues, I have intiatiated P2ILF challenge at the MICCAI 2022. I am also organising member of 1st Workshop on Cancer Prevention through early detecTion workshop at MICCAI 2022. I am a founding member of NAAMII, Kathmandu, Nepal and holds a volunteering adjunct research scientist position to train selected students from LMIC listed countries (more info here). I have supervised several research interns and currently supervising/co-supervising PhD students on topics around artificial intelligence for endoscopic, surgical vision and pathology:

Soumya Gupta (Co-supervisor; thesis submitted @ Univ. of Oxford)
Ziang Xu (Co-supervisor @ Univ. of Oxford)
Mansoor Ali (Co-supervisor @ Tec de Monterrey)
Bisi Bode Kolawole (Co-supervisor @ LSBU, London)
Gerardo Loza Galindo (Co-supervisor @ Univ. of Leeds)
Pedro-Chavarrias-Solano (primary supervisor @ Univ. of Leeds)
Supervising/supporting LMIC students (volunteer position at NAAMII, Nepal):

Ajay Pyatha 
Sweta Banerjee 
Mohammad Al-Amin 
I am actively involved in reviewiewing for several international journals (e.g., nature communication, IEEE TMI, Medical Image Analysis, Pattern Recognition..), and conferences (MICCAI, IEEE ISBI, IPCAI, ICPR..). I also serve as chair and panel member for international conferences and workshops. I have published in major journals and conferences including medical image analysis, pattern recognition, IEEE TNLLS, CVIU, Scientific reports, MICCAI conference, IEEE ISBI, IEEE EMBC, ICPR, and more. 

My recent podcast interview highlighting the need for multi-modality, multi-center approach for tackling generalisability and bias in AI for medical image analysis. 

Responsibilities
Level 1 Lead
Research interests
My research interest lies in solving computer vision and image processing related problems which includes building new mathematical models and implementation of the methods for both research purposes and practical use. I mostly work around applications in endoscopy, histology imaging, and other biomedical imaging. I am keen towards learning about new technologies and open to meeting new people, establishing interesting and innovative collaborations (both academia and industry), mentoring students and looking forward for new and exciting opportunities.

I have an extensive experience in building cutting edge computer vision technologies for object detection, semantic segmentation, image restoration, depth estimation, optical flow, image registration and 3D reconstruction. I have developed tools in these topics with both classical mathematical models and deep learning models. I am a passionate researcher with knowledge, energy and enthusiasm to contribute towards healthcare technologies that can benefit patients and assist clinicians around the world. I am a promotor of translational research and equality in access to digital healthcare for all. 

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Qualifications
PhD in medical image analysis from University of Lorraine, France
MSc in Computer Vision (by research), University of Dijon, France
Professional memberships
IEEE
MICCAI Society
IEEE EMBS
Student education
Artificial Intelligence

Deep Learning

 

Postgraduate research opportunities
We welcome enquiries from motivated and qualified applicants from all around the world who are interested in PhD study. Our research opportunities allow you to search for projects and scholarships."""

In [173]:
o = """Profile
I teach and research in Artificial Intelligence, Data Mining and Text Analytics, in the School of Computing at Leeds University. I also work 40% (2 days a week) at LITE Leeds Institute for Teaching Excellence as a LITE Fellow, on research in AI for Decolonizing Reading Lists. I am also a Turing Fellow at the Alan Turing Institute for Data Science and Artificial Intelligence and a member of LIDA Leeds Institute for Data Analytics and LATL Language At Leeds

I like 70s punk rock, eg … coming 2022 to Leeds:  11 Nov: London Calling play The Clash … 12 Nov: From The Jam … 12 Nov: The Buzzcocks … 6 Dec: The Cure.



I teach Artificial Intelligence, Data Mining and Text Analytics to stduents in several degree programmes: Computer Science MEng, BSc , Computer Science with Artificial Intelligence MEng, BSc , Computer Science and Mathematics MSci, BSc , Advanced Computer Science MSc , Advanced Computer Science (Data Analytics) MSc , Advanced Computer Science (Artificial Intelligence) MSc , Artificial Intelligence online part-time Masters / PGCert  , Data Science and Analytics MSc , Computing PhD , Artificial Intelligence for Medical Diagnosis and Care MSc and PhD ,  Leeds International Summer School  

I lead the AI4L Artificial Intelligence for Language research group, with

Co-Supervisors: Dr Noorhan Abbas, Dr Ammar Alsalka, Dr Abdulrahman Altahhan, Dr Joanna Leng, Dr Alison May, Dr Aisha Walker, Prof Seham Masoul (Princess Nourah University), Prof Abdulmalik Alsalman (King Saud University)

Research Fellows: Thomas Pickard, Luba Pirgova-Morgan

PhD research students:

Lamya Almaneea, due 2026, chatbots for university education
Rawan Bin Shiha, due 2026, Text Analytics and Decolonizing the Curriclum
Salwa Alahmari, due 2026, Detecting untrustworthy sources in social media
Alaa Alsaqer, due 2026, Comparing platforms for machine learning and text analytics
Bashaer Alsafari, due 2026, AI chatbots for university education
 Amaal Alhejaily, due 2026, Machine Learning of assessments for university education
Ibtisam Alshammari, due 2025, Knowledge base linking Quran ontologies
Sarah Alnefaie, due 2025, Artificial Intelligence for understanding Islamic texts
Sanaa Alowaidi, due 2025, knolwedge representation of Islamic texts
Albatool Alamri, due 2025, Cross-lingual computational stylometry
Saud Althabiti, due 2025, Detecting fake news in Arabic social media 
Moneerh Aleedy, due 2027, Chat and chatbots for university education
Abdullah Alsaleh, due 2024, Semantic similarity in Arabic text understanding
Shatha Altammami, due 2022, Parallel corpus and knowledge-base for Islamic Hadith
Taghreed Tarmom. due 2022, Classifying Arabic Hadith
Menwa Alshammeri, due 2022, Deep learning and distributional semantics for the Quran
Mashael Alamr, due 2022, Authorship Attribution and Online Identity in Najdi Arabic Tweets
J Alasamari, 2021, Arabic and English verb systems in the Quran corpus
N Ahmad, 2021, Malay natural language processing for retrieval from Malay Qur'an
A Alshutayri, 2019, Arabic dialects classification
L Aldhubayi, 2019, Corpus-based methods and to extend Arabic WordNet
A Alghamdi, 2018, Arabic corpus-informed lexicon of formulaic sequences
M Alqahtani, 2019, Quranic Arabic semantic search tool based on ontology of concepts
A Alosaimi, 2018, Ensemble morphosyntactic analyser for classical Arabic
S Alrehaili, 2018, Ontology concept and relation learning from the Qur’an corpus
J Jaafar, 2018, Data mining and machine learning to predict acute coronary illness
A Alfaifi, 2016, Arabic learner corpus and a system for Arabic error annotation
S Danso, 2016, Text analytics to predict time and cause of death from verbal autopsies
K Dukes, 2013, Statistical parsing by machine learning from a Classical Arabic treebank
S Hina, 2013, Semantic tagging of medical narratives using SNOMED CT
A Muhammad, 2012, Annotation of conceptual co-reference and similarity in the Qur'an
J Washtell, 2011, Distributional meaning in text: distance, expectation, and composition
M Sawalha, 2011, Open-source resources and standards for Arabic word structure analysis
C Brierley, 2011, Prosody resources for automated phrase break prediction
O Nancarrow, 2011, Tagging of adverbs in modern English corpora
F Su, 2010, Computational modelling of word sense sentiment
N Abbas, 2009, Qurany 'Search for a Concept' tool and website
A Roberts, 2008, Unsupervised machine learning for grammatical inference
D Elliott, 2006, Corpus-based machine translation evaluation via automated error detection
B AbuShawar, 2005, A corpus based approach to generalise a chatbot system
L Al-Sulaiti, 2004, Designing and developing a Corpus of Contemporary Arabic
T Oba, 2003, HTK to analyse prosody in the ISLE corpus of spoken learner's English
J Elliott, 2003, Natural language learning for SETI Search for Extra-Terrestrial Intelligence
X Duan, 2001, Lexical Semantic Association Between Web Documents
G Churcher, 1997, Speech dialogue analysis using linguistic knowledge
G Demetriou, 1997, Lexical semantics for human-computer speech communication.
M Schillo, 1996, Working while driving: corpus-based in-car personal assistant
X Zhang, 1996, MIRTH Chinese and English search engine: a multilingual retrieval tool
C Souter, 1996, A corpus-trained parser for systemic-functional syntax
A Bull, 1996, Aerobic dance exercise: a corpus-based computational linguistics approach
U Jost, 1994, Probabilistic language modeling for speech recognition
S Arnfield, 1994, Prosody and syntax in corpus-based analysis of spoken English
J Hughes, 1993, Automatically acquiring a classification of words
T O’Donoghue, 1993, Reversing the process of generation in Systemic Grammar
I am proud of over 60 research students and Research Fellows I have supervised, who went on to work in a range of careers, including web search, banking and finance, text analytics, translation and language consulting, online news, voice-to-text, the search for extra-terrestrial intelligence, and many university researchers and professors..

I live with my family and pets. 



Academic responsibilities: DPGRS Director of PostGraduate Resesrch Studies, Programme Leader for development of new online MSc in AI; plus: TEACHING: COMP2121 Data Mining c200 students), COMP5840M Data Mining and Text Analytics (c200 students), XJCO2121 Data Mining (c80 students), LISS1031 Data Mining and Text Analytics (c20 students), OCOM5204 Data Mining and Text Analytics (c40 students) OCOM5300 Artificial Intelligence Project (new module development for online MSc in AI) c16 BSc and MSc student research projects, c50 personal tutees; RESEARCH: Supervision: 13 PhD students, 2 Research Fellows; PLUS: writing publications, research project proposals etc; Peer Review College for AHRC, EPSRC, Qatar-QNRF, HongKong-RGC, Irish-RC, Poland-NCRD, EU-H2020; PhD and MSc external examiner; journal and conference reviewer; Arabic-L and AI4L research networks administrator.

My open-access publicatiions are available from Google Scholar, LinkedIn, Academia and ResearchGate, for anyone to read and cite … 

ResearchGate said:



Way to go, Eric!

With 1,532 new reads, your research items were the most read research items from your institution

Achieved on June 7, 2021

Gallup STRENGTHSFINDER found my top strengths are:  Learner, Achiever, Ideation, Intellection, Maximizer: I like learning, achieving, ideas, thinking, and excellence.

Previous academic responsibilities
2013 - 2017: Associate Professor, U of Leeds: Research, Teaching within AI group
1996 - 2013: Senior Lecturer, U of Leeds: Research, Teaching, NLP group leader
1994 - 1996: Director, JISC CALAS NTI: JISC UK R&D programme leader
1991 - 1994: National Coordinator, JISC KBSI: JISC UK R&D programme leader
1990 - 1991: SERC Advanced Research Fellow: SERC/MoD network coordinator
1984 - 1990: Lecturer, U of Leeds: Teaching, Research, leading NLP within AI team
1981 - 1984: Research Associate, University of Lancaster: Research, grant applications
Research Grants: Investigator
LITE Fellowship: AI for Decolonizing Reading Lists (LITE, £60K, 2022-2024
EDUBOTS chatbots for university education (ERASMUS+ £94K, 2019-2022)
FoodShock 2020 (Science and Technology Facilities Council, £8K, 2020-2021)
Artificial Intelligence Networking (Leeds University Academic Development Fund, £2K, 2017)
Natural Language Processing Working Together With Arabic And Islamic Studies (Engineering and Physical Science Research Council, £337K, 2013-2015)
Web-based resources for Islamic Studies (Higher Education Academy, £13K, 2011)
Detecting Terrorist Activities: Making Sense (Engineering and Physical Science Research Council, £2185K (£195K Leeds), 2010-2013)
Copying-Identifier for Biomedical Science Reports (Higher Education Funding Council, £60K, 2002)
ISLE - Interactive Spoken Language Education (European Union, £720K (£64K Leeds), 1998-2001)
Information extraction from air traffic control (Visionair International, £21K, 1994-1997)
Mapping between corpus annotation schemes (Nuffield Foundation, £6K, 1994)    
Computer based resources in KBS & SALT (Higher Education Funding Council, £92K, 1994-1996)
Knowledge based timetable information (Higher Education Funding Council, £39K, 1994-1995)
Mapping AMALGAM corpus (Science and Engineering Research Council, £176K, 1993-1997)
Computer analysis of language and speech (Higher Education Funding Council, £79K, 1993-1995)
Pilot project for CCALAS (Leeds University Research Support, £20K, 1993-1994)
A speech-oriented stochastic parser (Ministry of Defence, £178K, 1992-1993)
Knowledge-Based Systems Initiative (Joint Information Systems Committee, £149K, 1991-1994)
Neural network parsers trained with realistic corpora (British Telecom, £10K, 1990-1991)
Advanced Fellowship in IT (Science and Engineering Research Council, £220K, 1990-1994)
COMMUNAL Convivial man machine understanding (Ministry of Defence, £83K, 1987-1989)
Co-Investigator

Cancer patient records analysis (PinPoint, £52K, 2019-2020)
e-Health GATEway to the Clouds (Joint Information Systems Committee, £52K, 2012)
ABC Arabic By Computer (British Society for Middle Eastern Studies, £26K, 1989-1990)
A simulated annealing parser for authentic English (Ministry of Defence, £117K, 1986-1989)
Academic visits away from the University of Leeds
2018-2021: visiting professor, Leeds-SWJTU Joint School, SWJT University, Chengdu
2013 - 2017: visiting scholar, SUSTECH Sudan Uni of Science and Technology, Khartoum  
2012 - 2013: visiting scholar, Linguistics and English Language Dept, University of Manchester
2011 -2013: visiting scholar, Computing Dept, King Saud University, Riyadh, Saudi Arabia
2000: visiting scholar, Dept of Computer Science, University of Sheffield  
1994 - 1996: JISC Computer Analysis of Language And Speech New Technologies Initiative
1995: visiting scholar, Dept of Computer Applications, Dublin City University
1994: visiting scholar, Inst for Language & Artificial Intelligence, Tilburg University  
1991 - 1994: National Coordinator, JISC Knowledge Based Systems Initiative
1990 - 1991: Science and Engineering Research Council, Advanced Research Fellowship
1989: visiting scholar, Max Planck Institute and Dept of Language, Nijmegen University
Responsibilities
DPGR Director of PostGraduate Research studies
Programme Leader, online part-time MSc Artificial Intelligence
AI4L Artificial Intelligence for Language research leader
Research interests
My research speciality is Artificial Intelligence for Language research: AI applied in corpus linguistics and text analytics: Machine Learning and Data Mining analysis of a CORPUS of text - in English, Arabic, or other languages - to analyse the text and detect "interesting" and "useful" features or patterns. For example:

Understanding religious texts such as the Quran, to find links and patterns in the Quran verses and chapters whcih are of interest to religious scholars and the general public.
Detecting terrorist activities, by analysis of documents from terrorist suspects, to highlight suspicious parts of the text.
AI intelligent personal assistants for university students or lecturers, to advise students on courses, or write exam questions and answers from lecture transcripts, 
I have led research projects at Leeds University funded by research grants, including EU-funded EDUBOTS: chatbots for university education, EPSRC-funded Natural language processing working together with Arabic and Islamic Studies , EPSRC/ESRC/CPNI-funded IDEAS factory - detecting terrorist activities: making sense , JISC-funded e-Health GATEwayto the Clouds , HEA-funded web-based resources for teaching and research in Islamic Studies .

We are part of the Artificial Intelligence research theme of the School of Computing at the University of Leeds.  I also work 40% (2 days a week) at LITE Leeds Institute for Teaching Excellence as a LITE Fellow, on research in AI for Decolonizing Reading Lists. I am also a Turing Fellow at the Alan Turing Institute for Data Science and Artificial Intelligence and a member of LIDA Leeds Institute for Data Analytics and LATL Language At Leeds

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Qualifications
PhD (Leeds) Corpus linguistics and language learning, 2008 http://etheses.whiterose.ac.uk/7504/
BA 1st Class (Lancaster) Computing and Linguistics, 1981
Professional memberships
EPSRC (Engineering and Physical Science Research Council) Peer Review College
AHRC (Arts and Humanties Research Council) Peer Review College
Student education
My teaching includes: supervision of PhD, MSc, MEng and BSc student research projects; lecturing to Computing MSc, MEng and BSc students; and personal tutorials for students. I have also taught at international conference tutorials and summer schools, for example Summer School on Corpus Linguistics. 

I have taught a range of subjects, including Corpus Linguistics, Data Mining, Text Analytics, Language, Natural Language Processing, Computational Linguistics, Knowledge Management and Adaptive Systems, Technologies for Knowledge Management, Object Oriented Programming, Professional Development, Knowledge Based Systems, Knowledge Discovery, Perceptual Systems, Future Directions in Distributed Multimedia Systems, Introductory Programming, Numerical Methods, Artificial Intelligence, Databases and Information Systems, Arabic Natural Language Processing, Java, Python, Prolog, Pascal.

I teach stduents in several degree programmes: Computer Science MEng, BSc , Computer Science with Artificial Intelligence MEng, BSc , Computer Science and Mathematics MSci, BSc , Advanced Computer Science MSc , Advanced Computer Science (Data Analytics) MSc , Advanced Computer Science (Artificial Intelligence) MSc , Artificial Intelligence online part-time Masters / PGCert  , Data Science and Analytics MSc , Computing PhD , Artificial Intelligence for Medical Diagnosis and Care MSc and PhD ,  Leeds International Summer School  

 
Student feedback:
“He is a fantastic lecturer who has made the semester significantly more enjoyable with his live lectures. With two semesters which were otherwise ruined by covid-19, COMP2121 was genuinely enjoyable to watch. Plus he has a lovely kitten which made some guest appearances in lecture, which alone should qualify him for an award.”
“i love the commitment and enthusiasm of the module leader Eric Atwell”
“The module leader was lovely and so was his kitten. The coursework was interesting and the group work was helpful. There were a lot of learning materials provided”
“He has been really supportive and understanding  during this unprecedented time. Also he helps his students develop other soft skills in addition to their intellectual development.”
“The content was interesting and the lecturer was engaged with the topic”
“I highly recommend this course. The class content of this course is rich, and the teaching methods are diversified, which can well apply the theory to practice”
“He is supportive and encourage me every step of the way”
“I learnt a lot from you, many thanks for your regular excellent supervision, caring, and the considerable advices you provided to me, patiently listened to me in meetings, and gave me the best suggestions. Thank you for your endless help, support and guidance.”
“Liked the videos Eric would play before the lecture started on Friday mornings!”
“Eric has made me feel like I am attending lectures to learn and not just to pass an exam. When other areas of our course have got tough he has postponed exams and tried to make them as doable as possible for his students. He also makes regular contact with his students and is good at what he does.”
“I thought that the content taught in the module was interesting and engaging, and we were provided with lots of supplementary papers and resources to support our learning”
“Thank you, because you had been a great master for me. A teacher who does not tire of my questions, Support when facing problems, and full of patience.”
“Professor Eric is very cooperative with his students and is always available to assist them and answer their inquiries. He always tries to involve them in research groups and reading groups to expand their knowledge. Also, he is committed to time and meeting schedules.”
“Eric is very clearly passionate about the subject, and that's something that always shines through to students. He's an engaging teacher and is more than happy to help with any questions”
“Liked that the module contained a high percentage of coursework content (rather than just exams).”
“Eric Atwell has been very supportive of my PhD studies since the start of my journey. He pushed me to read papers and experiment with models, and then publish papers once I successfully finish with experiments. He also reviews my draft papers and reports that the faculty is required me to submit. Eric has also been in contact to his students in regards to finding opportunities to grow as a researcher. Finally, he has never missed or cancelled any meetings throughout my journey which is something that tells me that he is with me along the way.”
“Lecture content was interesting and well presented”
“The teacher fully integrates his own research with the curriculum, and exercises the students' teamwork ability”
“I've always enjoyed Eric's lectures because he seemed to genuinely want us to enjoy his module and learn from it. He also put effort in to reduce our stress and be able to actually enjoy the module and want to work for it, opposed to just pass a test.”
“The content was interesting and the professor was really nice and enthusiastic”
“really amiable lecturer, really easy to understand lecturer”
“Nice that there's lots of hands-on/demonstration of material and approaches”
“Enjoyed it as it provided a range of academic enlightenment in the field”
“The teaching method was really impressive, professor also introduced some of the new research ongoing in this field.”
Research groups and institutes
Artificial Intelligence
Current postgraduate researchers
Menwa Alshammeri
Shatha Altammami
Taghreed Tarmom
Lena Almutair
Abdullah Alsaleh
Moneerh Aleedy
Ibtisam Alshammari
Saud Althabiti
Albatool Alamri
Sarah Alnefaie
Bashaer Alsafari
Alaa Alsaqer
Sanaa Alowaidi
Salwa Alahmari
Rawan Bin Shiha
Amaal Alhejaily
Postgraduate research opportunities
We welcome enquiries from motivated and qualified applicants from all around the world who are interested in PhD study. Our research opportunities allow you to search for projects and scholarships.
"""

In [46]:
generate_summary_brat(y)

'Profile of me:Third-year computer science undergraduate student and teaching assistant at the University of Leeds. I am the founder and director of my own company OC Solutions Ltd, providing bespoke information technology and software solutions and consultancy.Profile of myself:Professionals:Computer Science, Computer Science, Engineering and Technology.Responsibilities:Undergraduate Teaching Assistant.Research interests:I have completed my dissertation on Survival Analysis with Neural Networks for Chronic Heart Failure Patients.Research projects:Research projects'

In [47]:
generate_summary_textRank(y)

'Professional memberships\nStudent Membership - BCS, The Chartered Institute for IT\nStudent Membership - IET, Institution of Engineering and Technology\nStudent education\nIn Semester 1, I mainly aided Level 1 students in lab sessions, focusing on the Introduction to Programming and Procedural Programming modules.'

In [22]:


print(api.info())

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1, 'record_format': 'dict', 'file_size': 6344358, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py', 'license': 'All files released for the task are free for general research use', 'fields': {'2016-train': ['...'], '2016-dev': ['...'], '2017-test': ['...'], '2016-test': ['...']}, 'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.', 'checksum': '701ea67acd82e75f95e1d8e62fb0ad29', 'file_name': 'semeval-2016-2017-task3-subtaskBC.gz',

In [50]:
model_name = "word2vec-google-news-300"  # 模型名称
model_name2 = "fasttext-wiki-news-subwords-300"
model_fasttext = api.load(model_name2)

In [65]:
df['text'][0]

'i teach and research in artificial intelligence python for data science data science data mining and text analytics in the school of computing at university of leeds  i worked in the  project which is an erasmus knowledge alliance project that aims to improve results and raise attainment levels in european higher education institutions this project focussed on exploring best practices and innovative use of chatbots and creating a learning community of educators i have experience in organising conferences and workshops  i organised two conferences for the  project in 2023 i organised the turing artificial intelligence data science and text analytics for education resources workshop ai4edures all the video recordings and powerpoint slides for the ai4edures workshop could be found here i am also a member of the lida leeds institute for data analytics research funding alan turing institute  2022 postdoctoral enrichment award  2022pea100083                                                re

In [201]:
# 导师介绍文字列表
# tutor_texts = [
#     "I like reading books.",
#     "I like playing piano.",
#     "I like driving.",
# ]

tutor_texts = df['text'].tolist()
print(tutor_texts.__len__())

# 加载预训练的词向量模型
word2vec_model = model_fasttext


# 文本表示
tutor_vectors = []
for text in tutor_texts:
    text_tokens = text.split()  # 将文本分词为单词列表
    vector = []
    for token in text_tokens:
        if token in word2vec_model.key_to_index:
            vector.append(word2vec_model[token])
    if vector:
        vector = np.mean(vector, axis=0)  # 计算词向量的平均值
        tutor_vectors.append(vector)

# 用户输入的文本
user_text = "want to learn text analysis and data mining."

user_vector_tokens = user_text.split()
user_vector = []
for token in user_vector_tokens:
    if token in word2vec_model.key_to_index:
        user_vector.append(word2vec_model[token])
if user_vector:
    user_vector = np.mean(user_vector, axis=0)

# 计算相似度
similarities = cosine_similarity([user_vector], tutor_vectors)[0]

# 推荐导师
recommended_tutor_index = similarities.argmax()
recommended_tutor = df['Name'][recommended_tutor_index]

print("推荐导师：", recommended_tutor)


14
推荐导师： Hamish Carr


In [129]:
df['text'] = df['text'].str.replace('amunts', '')